In [3]:
import pandas as pd
import re
import numpy as np
import codecs
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') 
from nltk.tokenize import word_tokenize
set_words = set(stopwords.words('english'))
tokenizer=nltk.tokenize.RegexpTokenizer('\w+')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from scipy.sparse import hstack
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20109\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df=pd.read_csv(r"B:\Electronics And Electrical COMM\Courses\Music Recomm\reducedfile.csv")

In [5]:
df.shape

(200000, 11)

In [6]:
df = df.dropna(how='all', subset=['language', 'language_cld3', 'language_ft'])

In [7]:
df.shape

(198385, 11)

In [8]:
def remove_curly_braces(text):
    return text.replace('{', '').replace('}', '').replace('"','')


df['features'] = df['features'].apply(remove_curly_braces)

In [9]:
languages = df['language_cld3'].unique()

In [10]:
all_languages = set(df['language_cld3'].dropna().tolist() + 
                    df['language_ft'].dropna().tolist() + 
                    df['language'].dropna().tolist())

# Create new columns for each language
for lang in all_languages:
    # Create a column name for the language
    col_name = f'lang_{lang}'
    
    # Create a new column filled with zeros
    df[col_name] = 0
    
    # Update the column value based on presence in each language-related column
    df.loc[df['language_cld3'].str.contains(lang, case=False, na=False), col_name] = 1
    df.loc[df['language_ft'].str.contains(lang, case=False, na=False), col_name] = 1
    df.loc[df['language'].str.contains(lang, case=False, na=False), col_name] = 1
tags=set(df['tag'].dropna().tolist())

for tag in tags:
    # Create a column name for the language
    col_name = f'tag_{tag}'
    
    # Create a new column filled with zeros
    df[col_name] = 0
    
    df.loc[df['tag'].str.contains(tag, case=False, na=False), col_name] = 1

C:\Users\20109\AppData\Local\Temp\ipykernel_21280\159669406.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = 0
C:\Users\20109\AppData\Local\Temp\ipykernel_21280\159669406.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = 0
C:\Users\20109\AppData\Local\Temp\ipykernel_21280\159669406.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1)

In [11]:
dfx=df[df['language_cld3']!=df['language_ft']]

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,lang_id,lang_yi,lang_sl,lang_ca,lang_ku,lang_oc,lang_cs,lang_sd,lang_ka,lang_ha,lang_ckb,lang_ky,lang_zh,lang_arz,lang_et,lang_mg,lang_gl,lang_tt,lang_ta,lang_mt,lang_uz,lang_tg,lang_als,lang_my,lang_eo,lang_ig,lang_ny,lang_vi,lang_fa,lang_nl,lang_lb,lang_ja,lang_fy,lang_hi,lang_fr,lang_bn,lang_eu,lang_uk,lang_it,lang_sw,lang_no,lang_gu,lang_si,lang_da,lang_zu,lang_ur,lang_pa,lang_sv,lang_ms,lang_bg,lang_pl,lang_lt,lang_xh,lang_ceb,lang_sm,lang_so,lang_te,lang_ko,lang_br,lang_el,lang_mn,lang_sq,lang_st,lang_he,lang_hu,lang_la,lang_ru,lang_sk,lang_sah,lang_ps,lang_es,lang_sr,lang_af,lang_mi,lang_mk,lang_hy,lang_nn,lang_jv,lang_bs,lang_ht,lang_am,lang_de,lang_ar,lang_hr,lang_fil,lang_haw,lang_sn,lang_ro,lang_th,lang_cy,lang_nds,lang_tr,lang_lv,lang_jbo,lang_ga,lang_pt,lang_fi,lang_gd,lang_km,lang_be,lang_yo,lang_mr,lang_ne,lang_su,lang_kk,lang_en,lang_az,lang_is,lang_co,tag_misc,tag_country,tag_pop,tag_rap,tag_rock,tag_rb
0,Complaints,pop,Somatina,2019,68,,[Verse 1]\nI don't want to look like a frustra...,5133768,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,12AM,rap,Lil Macks,2021,6443,,"[Chorus]\nIt's 12am, I found myself in this ri...",7297758,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,Fantastic,rap,LMEN PRALA,2021,2027,Curtis,"[Refrén]\nMég táncolj, rázzad, ne parázz bae\n...",7073645,hu,hu,hu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,VIOLENT 3,rap,UG Planet,2021,575,"Fairy5100,Ayomagg,LuvLilGeo,Raidurr,Kidd Akimb...","[Intro]\n(Goated, this shit is crazy)\n\n[Vers...",7274717,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,Snow White,pop,Zack Martino,2021,41,Kyle Reynolds,"[Verse 1]\nShe don't need money, she likes to ...",7188507,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [13]:
data=df

In [14]:
data.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,lang_id,lang_yi,lang_sl,lang_ca,lang_ku,lang_oc,lang_cs,lang_sd,lang_ka,lang_ha,lang_ckb,lang_ky,lang_zh,lang_arz,lang_et,lang_mg,lang_gl,lang_tt,lang_ta,lang_mt,lang_uz,lang_tg,lang_als,lang_my,lang_eo,lang_ig,lang_ny,lang_vi,lang_fa,lang_nl,lang_lb,lang_ja,lang_fy,lang_hi,lang_fr,lang_bn,lang_eu,lang_uk,lang_it,lang_sw,lang_no,lang_gu,lang_si,lang_da,lang_zu,lang_ur,lang_pa,lang_sv,lang_ms,lang_bg,lang_pl,lang_lt,lang_xh,lang_ceb,lang_sm,lang_so,lang_te,lang_ko,lang_br,lang_el,lang_mn,lang_sq,lang_st,lang_he,lang_hu,lang_la,lang_ru,lang_sk,lang_sah,lang_ps,lang_es,lang_sr,lang_af,lang_mi,lang_mk,lang_hy,lang_nn,lang_jv,lang_bs,lang_ht,lang_am,lang_de,lang_ar,lang_hr,lang_fil,lang_haw,lang_sn,lang_ro,lang_th,lang_cy,lang_nds,lang_tr,lang_lv,lang_jbo,lang_ga,lang_pt,lang_fi,lang_gd,lang_km,lang_be,lang_yo,lang_mr,lang_ne,lang_su,lang_kk,lang_en,lang_az,lang_is,lang_co,tag_misc,tag_country,tag_pop,tag_rap,tag_rock,tag_rb
0,Complaints,pop,Somatina,2019,68,,[Verse 1]\nI don't want to look like a frustra...,5133768,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,12AM,rap,Lil Macks,2021,6443,,"[Chorus]\nIt's 12am, I found myself in this ri...",7297758,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,Fantastic,rap,LMEN PRALA,2021,2027,Curtis,"[Refrén]\nMég táncolj, rázzad, ne parázz bae\n...",7073645,hu,hu,hu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,VIOLENT 3,rap,UG Planet,2021,575,"Fairy5100,Ayomagg,LuvLilGeo,Raidurr,Kidd Akimb...","[Intro]\n(Goated, this shit is crazy)\n\n[Vers...",7274717,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,Snow White,pop,Zack Martino,2021,41,Kyle Reynolds,"[Verse 1]\nShe don't need money, she likes to ...",7188507,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [15]:
data.loc[3]['lyrics']

'[Intro]\n(Goated, this shit is crazy)\n\n[Verse 1: LuvLilGeo]\nYeah\nViolent the third, if you talkin\' on me, you get hurt (Yeah, huh)\nWe put \'em up on a tee and we put \'em up on a shirt (Yeah, huh)\nImma ride \'round with that semi and Imma ride \'round with that bird (Yeah huh)\nImma ride round with that, uh, you fucking wit me you get murked (Yeah huh)\nMovin’ in silence we moving in violence\nI stay with my narcos we stay in the night\nS-s-slide with that chopper\nI slide up with vino\nWe slide on yo momma, then shoot in the eye\nStay wit my gang and I stay wit my slime\nAnd I know I said once, but I\'m saying it twice\nI said it once and I\'m saying it twice\nPussy ass boy yеah you know we gone slide, yеah\nViolent the third, you can get hurt\nFuck around me pussy boy you get murked (yeah)\nI don\'t be doing no shit to these niggas, but if you want smoke, then lil bitch you could get it\nS-s-stay with my gang if you talk down on vino, I slide wit that choppa, then shoot at yo

In [16]:
def remove_patterns_within_brackets(text):
    pattern = r'\[[^\]]+\]'
    return re.sub(pattern, '', text)
data['lyrics']=data['lyrics'].apply(remove_patterns_within_brackets)

In [17]:
data.head()

data2=data
data2['artist']=data2['artist'].str.replace(' ','')
data2['features']=data2['features'].str.replace(' ','')
data2.head(10)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,lang_id,lang_yi,lang_sl,lang_ca,lang_ku,lang_oc,lang_cs,lang_sd,lang_ka,lang_ha,lang_ckb,lang_ky,lang_zh,lang_arz,lang_et,lang_mg,lang_gl,lang_tt,lang_ta,lang_mt,lang_uz,lang_tg,lang_als,lang_my,lang_eo,lang_ig,lang_ny,lang_vi,lang_fa,lang_nl,lang_lb,lang_ja,lang_fy,lang_hi,lang_fr,lang_bn,lang_eu,lang_uk,lang_it,lang_sw,lang_no,lang_gu,lang_si,lang_da,lang_zu,lang_ur,lang_pa,lang_sv,lang_ms,lang_bg,lang_pl,lang_lt,lang_xh,lang_ceb,lang_sm,lang_so,lang_te,lang_ko,lang_br,lang_el,lang_mn,lang_sq,lang_st,lang_he,lang_hu,lang_la,lang_ru,lang_sk,lang_sah,lang_ps,lang_es,lang_sr,lang_af,lang_mi,lang_mk,lang_hy,lang_nn,lang_jv,lang_bs,lang_ht,lang_am,lang_de,lang_ar,lang_hr,lang_fil,lang_haw,lang_sn,lang_ro,lang_th,lang_cy,lang_nds,lang_tr,lang_lv,lang_jbo,lang_ga,lang_pt,lang_fi,lang_gd,lang_km,lang_be,lang_yo,lang_mr,lang_ne,lang_su,lang_kk,lang_en,lang_az,lang_is,lang_co,tag_misc,tag_country,tag_pop,tag_rap,tag_rock,tag_rb
0,Complaints,pop,Somatina,2019,68,,\nI don't want to look like a frustrated girl\...,5133768,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,12AM,rap,LilMacks,2021,6443,,"\nIt's 12am, I found myself in this ride with ...",7297758,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,Fantastic,rap,LMENPRALA,2021,2027,Curtis,"\nMég táncolj, rázzad, ne parázz bae\nÚgy, hog...",7073645,hu,hu,hu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,VIOLENT 3,rap,UGPlanet,2021,575,"Fairy5100,Ayomagg,LuvLilGeo,Raidurr,KiddAkimbo...","\n(Goated, this shit is crazy)\n\n\nYeah\nViol...",7274717,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,Snow White,pop,ZackMartino,2021,41,KyleReynolds,"\nShe don't need money, she likes to roll it u...",7188507,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
5,Khoone,rap,Sadegh,2015,73,,چه حس خوبیه تُو بارون قدم زدن\n\nدوست دارم هیچ...,4573626,fa,fa,fa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,ON SIGHT,rap,Guddabam,2019,30,Esjxnky,Stitch lips ops coming we don’t make sounds\nB...,4688563,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
7,Slow Down,pop,JesseDee,2008,828,,Somehow it got in our heads\nTo go as far as w...,2245857,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
8,METADON,rap,PGxDRINK,2021,23,,"\nТука знаеш кво ше стане\nBaby, споко, споко\...",7689521,bg,bg,bg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
data2['text_features'] = (
    data['artist'] + ' ' +  
    data['features'] + ' ' + 
    data['features'] + ' ' +  
    data['artist'] + ' ' +     
    data['artist'] + ' ' +  
    data['tag'] + ' ' + 
    data['tag'] + ' ' + 
    data['title'] + ' ' + 
    data['lyrics'] + ' ' + 
    data['title'] + ' ' + 
    data['artist'] + ' '+
    data['tag'] + ' ' + 
    data['tag'] + ' ' + 
    data['features'] + ' ' +
    data['features'] + ' ' +  
    data['artist'] + ' ' +
    data['artist'] + ' ' +     
    data['artist'] + ' ' +  
    data['artist']
    
)

C:\Users\20109\AppData\Local\Temp\ipykernel_21280\3339178200.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data2['text_features'] = (


In [19]:
data2.head()
data3=data2

In [20]:
data3['text_features'].fillna('', inplace=True)
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text features
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text_features'])


In [21]:
print(tfidf_matrix)

  (0, 488921)	0.044796943725714515
  (0, 494254)	0.06672423751727909
  (0, 436728)	0.04946727834983188
  (0, 216815)	0.03667129432166178
  (0, 644368)	0.058742453528580055
  (0, 421827)	0.03412115717536138
  (0, 551968)	0.09921302756887482
  (0, 609833)	0.028691947602675988
  (0, 949003)	0.08824070562788493
  (0, 564163)	0.05525029534835508
  (0, 513905)	0.042719492723127465
  (0, 278747)	0.035715603771137465
  (0, 22728)	0.027641363136572485
  (0, 481302)	0.02836660411680332
  (0, 495023)	0.01993510504759592
  (0, 863638)	0.023453040075875408
  (0, 122133)	0.04772055845788124
  (0, 529870)	0.012304083409697646
  (0, 160008)	0.021883636640378516
  (0, 955632)	0.02163633442501967
  (0, 402421)	0.05329286836759059
  (0, 56783)	0.0347957123459419
  (0, 206787)	0.032549439945820914
  (0, 860762)	0.04285028229821605
  (0, 835078)	0.038664590017235156
  :	:
  (198384, 766512)	0.016749756460618447
  (198384, 976053)	0.06466700645883453
  (198384, 949599)	0.11535237022311248
  (198384, 216815)

In [23]:
def recommendation(IDS, df):
    texts = df[df['id'].isin(IDS)]['text_features']
    
   
    input_tfidf_matrix = tfidf_vectorizer.transform(texts)
    
    similarities = cosine_similarity(input_tfidf_matrix, tfidf_matrix)
    
    
    average_similarity = np.mean(similarities, axis=0)
    
    
    sorted_indices = average_similarity.argsort()[::-1]
    
    recommended_artists = df.iloc[sorted_indices[0:20]]
    
    return recommended_artists
     

In [24]:
songs_recommended=recommendation([7539518,124418,3322893,2225459,3402550,1498,7476506],data3)

In [25]:
songs_recommended.head(10)


,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,lang_id,lang_yi,lang_sl,lang_ca,lang_ku,lang_oc,lang_cs,lang_sd,lang_ka,lang_ha,lang_ckb,lang_ky,lang_zh,lang_arz,lang_et,lang_mg,lang_gl,lang_tt,lang_ta,lang_mt,lang_uz,lang_tg,lang_als,lang_my,lang_eo,lang_ig,lang_ny,lang_vi,lang_fa,lang_nl,lang_lb,lang_ja,lang_fy,lang_hi,lang_fr,lang_bn,lang_eu,lang_uk,lang_it,lang_sw,lang_no,lang_gu,lang_si,lang_da,lang_zu,lang_ur,lang_pa,lang_sv,lang_ms,lang_bg,lang_pl,lang_lt,lang_xh,lang_ceb,lang_sm,lang_so,lang_te,lang_ko,lang_br,lang_el,lang_mn,lang_sq,lang_st,lang_he,lang_hu,lang_la,lang_ru,lang_sk,lang_sah,lang_ps,lang_es,lang_sr,lang_af,lang_mi,lang_mk,lang_hy,lang_nn,lang_jv,lang_bs,lang_ht,lang_am,lang_de,lang_ar,lang_hr,lang_fil,lang_haw,lang_sn,lang_ro,lang_th,lang_cy,lang_nds,lang_tr,lang_lv,lang_jbo,lang_ga,lang_pt,lang_fi,lang_gd,lang_km,lang_be,lang_yo,lang_mr,lang_ne,lang_su,lang_kk,lang_en,lang_az,lang_is,lang_co,tag_misc,tag_country,tag_pop,tag_rap,tag_rock,tag_rb,text_features
132651,3azra2eel | عزرائيل,rap,Abyusif,2017,89435,,\nبالإكتئاب مرتبط زي الموضة وسعر الشيء\nهلاوس ...,3402550,ar,ar,ar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Abyusif Abyusif Abyusif rap rap 3azra2eel | ...
166662,BIG MAFIA | بيج مافيا,rap,MarwanMoussa,2021,8988,,\nإيه إيه مافيا\nبيج مود، بيج مافيا\nبيج مود، ...,7539518,ar,ar,ar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,MarwanMoussa MarwanMoussa MarwanMoussa rap r...
7826,Katr Min Orbak | كتر من قربك,pop,AmrDiab,2021,515,,\nيا حبيبي إتمختر، إنت الأجمل إنت الأخطر\nدا إ...,7476506,ar,ar,ar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,AmrDiab AmrDiab AmrDiab pop pop Katr Min Orb...
56754,VIZA | ڤيزا,rap,Abyusif,2013,1648,,\nظا ظو ظي ظا ظو\nرازوا دوبي في بار نو\nزاروا ...,3322893,ar,ar,ar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Abyusif Abyusif Abyusif rap rap VIZA | ڤيزا ...
55125,5 AM in Toronto,rap,Drake,2013,1486962,,\nYeah\nThis on some old Tommy Campos Dice Raw...,124418,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,Drake Drake Drake rap rap 5 AM in Toronto \n...
9839,Juice,rap,Drake,2009,34650,,\n\n\nBoi-1da... Drizzy\n\n\nY'all don't reall...,1498,en,en,en,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,Drake Drake Drake rap rap Juice \n\n\nBoi-1d...
83218,Wayah | وياه,pop,AmrDiab,2009,2357,,\nده اللي كان نفسي فيه\nلو تيجي صدفة تجمعني بي...,2225459,ar,ar,ar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,AmrDiab AmrDiab AmrDiab pop pop Wayah | وياه...
142660,That Thing | الشىء,rap,Arsenik(EG),2019,16626,,\nمحدش ليه فضل عليا في الشيء ده، أنا بدأته لوح...,4490380,ar,ar,ar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0